## Supervised models
This notebook is intended for giving an introduction the ML supervised models that can be used for Covid detection.

For this notebook to find the new modules created for this project, we need to set its path to be in the root directory.

In [1]:
import sys
sys.path.append("../")

## Loading packages and dependencies

In [2]:
import numpy as np

from src.features.extract_features import load_extracted_features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from src.models.build_model import train_advanced_supervised_model, evaluate_model


# Path to the raw data and preprocessed data
raw_data_dir = '../data/raw/COVID-19_Radiography_Dataset/'
IMG_SIZE = 299  # Resize images to IMG_SIZExIMG_SIZE pixels

## Extracting features from images

In [3]:
F_healthy, y_healthy, X_healthy = load_extracted_features(raw_data_dir+'{}/images','NORMAL', 0, IMG_SIZE, image_resized=True)
F_sick, y_sick, X_sick = load_extracted_features(raw_data_dir+'{}/images',['COVID','Viral Pneumonia','Lung_Opacity'], 1, IMG_SIZE, image_resized=True)

Loaded images for NORMAL: 10192 resized images, 10192 features, and 10192 labels.
Loaded images for ['COVID', 'Viral Pneumonia', 'Lung_Opacity']: 10973 resized images, 10973 features, and 10973 labels.


## Normalizing features

In [4]:
# Combine datasets
# Keep 7 features only
# F_healthy = [f[:7] for f in F_healthy]
# F_sick = [f[:7] for f in F_sick]

F = np.vstack((F_healthy, F_sick)) #image features
X = np.vstack((X_healthy, X_sick)) #image data
y = np.concatenate((y_healthy, y_sick)) #labels

X_img_train, X_img_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X, F, y, test_size=0.2, random_state=42, stratify=y
)
print(f"X_train shape: {X_feat_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_feat_test.shape}, y_test shape: {y_test.shape}")
print(f"X_img_train shape: {X_img_train.shape}")

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_feat_train)
X_test = scaler.transform(X_feat_test)
X_feat_train = scaler.fit_transform(X_feat_train)
X_feat_test = scaler.transform(X_feat_test)

X_train shape: (16932, 14), y_train shape: (16932,)
X_test shape: (4233, 14), y_test shape: (4233,)
X_img_train shape: (16932, 299, 299)


In [5]:
# Reshape image data for CNN
X_img_train = X_img_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X_img_test = X_img_test.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

print(f"X_img_train shape: {X_img_train.shape}")
print(f"X_img_test shape: {X_img_test.shape}")

X_img_train shape: (16932, 299, 299, 1)
X_img_test shape: (4233, 299, 299, 1)


## Training and evaluating models

### Convolutional Neural Networks (CNN)

✅ Strengths:
* Highly accurate for image tasks.
* Learns complex patterns automatically.
* Works well with large image datasets.

❌ Weaknesses:
* Computationally expensive (needs GPUs).
* Requires large labeled datasets.
* Not easily interpretable.

In [6]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10)

2025-02-02 16:07:19.867123: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2025-02-02 16:07:19.867142: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 128.00 GB
2025-02-02 16:07:19.867147: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 48.00 GB
2025-02-02 16:07:19.867162: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-02-02 16:07:19.867171: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 299, 299,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 149, 149,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 149, 149,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 74, 74,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 74, 74,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 37, 37,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │        480 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 175232)    │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 175264)    │          0 │ flatten[0][0],    │
│ (Concatenate)       │                   │            │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │ 11,216,960 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,310,177 (43.14 MB)

 Trainable params: 11,310,177 (43.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


2025-02-02 16:07:22.812550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1059/1059 ━━━━━━━━━━━━━━━━━━━━ 41s 37ms/step - accuracy: 0.6550 - loss: 0.8559 - val_accuracy: 0.8068 - val_loss: 0.4324
Epoch 2/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.7692 - loss: 0.5308 - val_accuracy: 0.7938 - val_loss: 0.4779
Epoch 3/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7973 - loss: 0.4804 - val_accuracy: 0.8193 - val_loss: 0.5636
Epoch 4/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7732 - loss: 0.7365 - val_accuracy: 0.8162 - val_loss: 0.9039
Epoch 5/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 37s 35ms/step - accuracy: 0.7656 - loss: 1.4572 - val_accuracy: 0.7382 - val_loss: 3.8682
Epoch 6/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 38s 36ms/step - accuracy: 0.7563 - loss: 5.9334 - val_accuracy: 0.8155 - val_loss: 5.0635
Epoch 7/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7390 - loss: 26.2425 - val_accuracy: 0.8131 - val_loss: 57.7446
Epoch 8/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 38s 36ms/step - accuracy: 0.7249 - loss: 17

In [7]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="CNN")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")


Train Accuracy: 0.7092, Train Loss: 1768.6885
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7690 - loss: 1469.8152
Test Accuracy: 0.7690, Test Loss: 1461.3030


### Capsule Network

✅ Strengths:

* CapsNet learns spatial relationships better than CNNs.
* More robust to rotation & deformation in medical images.
* Less training data required but computationally expensive.

#### RBF kernel

In [8]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10, model_type="Capsule Network")

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 299, 299,  │        640 │ input_layer_2[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 299, 299,  │     18,464 │ conv2d_3[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 2860832)   │          0 │ conv2d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 32, 89401) │          0 │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32, 128)   │ 11,443,456 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 32)        │        480 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 4096)      │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 4128)      │          0 │ flatten_2[0][0],  │
│ (Concatenate)       │                   │            │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 64)        │    264,256 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 64)        │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1)         │         65 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 11,727,361 (44.74 MB)

 Trainable params: 11,727,361 (44.74 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 71s 66ms/step - accuracy: 0.6480 - loss: 0.7623 - val_accuracy: 0.7723 - val_loss: 0.7764
Epoch 2/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 74s 70ms/step - accuracy: 0.6812 - loss: 2.0711 - val_accuracy: 0.7756 - val_loss: 1.8550
Epoch 3/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 71s 67ms/step - accuracy: 0.6654 - loss: 7.8044 - val_accuracy: 0.7864 - val_loss: 8.0277
Epoch 4/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 68s 64ms/step - accuracy: 0.6887 - loss: 30.0880 - val_accuracy: 0.7999 - val_loss: 29.1838
Epoch 5/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 66s 62ms/step - accuracy: 0.6945 - loss: 99.5408 - val_accuracy: 0.7912 - val_loss: 74.6851
Epoch 6/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 65s 61ms/step - accuracy: 0.7047 - loss: 170.3077 - val_accuracy: 0.7893 - val_loss: 109.8907
Epoch 7/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 64s 60ms/step - accuracy: 0.6935 - loss: 258.0522 - val_accuracy: 0.7980 - val_loss: 110.7315
Epoch 8/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 63s 60ms/step - accura

In [9]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="Capsule Network")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

Train Accuracy: 0.6676, Train Loss: 6.4810
133/133 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.6101 - loss: 4.7443
Test Accuracy: 0.6119, Test Loss: 4.6779


### Transfer learning

✅ Strengths
* Transfer learning reduces training time while maintaining high accuracy.
* Fine-tuning improves performance when sufficient data is available.
* Combining deep features with statistical features can enhance results.

In [10]:
model, history = train_advanced_supervised_model([X_img_train, X_feat_train], y_train, [X_img_test, X_feat_test], y_test, 
                                [IMG_SIZE,X_feat_train.shape[1]], 10, model_type="Transfer Learning")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ input_layer_4[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 299, 299,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 299, 299,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        512 │ block1a_se_excit

 Total params: 4,224,388 (16.11 MB)

 Trainable params: 174,817 (682.88 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

Epoch 1/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 70s 62ms/step - accuracy: 0.5887 - loss: 0.6971 - val_accuracy: 0.6636 - val_loss: 0.6209
Epoch 2/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 61s 58ms/step - accuracy: 0.6366 - loss: 0.6637 - val_accuracy: 0.6995 - val_loss: 0.6159
Epoch 3/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 61s 58ms/step - accuracy: 0.6551 - loss: 0.6503 - val_accuracy: 0.6924 - val_loss: 0.6108
Epoch 4/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 61s 58ms/step - accuracy: 0.6506 - loss: 0.6508 - val_accuracy: 0.6979 - val_loss: 0.6087
Epoch 5/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 62s 58ms/step - accuracy: 0.6601 - loss: 0.6497 - val_accuracy: 0.7059 - val_loss: 0.6088
Epoch 6/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 63s 59ms/step - accuracy: 0.6582 - loss: 0.6511 - val_accuracy: 0.7009 - val_loss: 0.6045
Epoch 7/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 63s 59ms/step - accuracy: 0.6548 - loss: 0.6769 - val_accuracy: 0.7078 - val_loss: 0.6076
Epoch 8/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 63s 59ms/step - accuracy: 0.6539 -

In [11]:
train_loss, train_acc = history.history['loss'][-1], history.history['accuracy'][-1]
print(f"Train Accuracy: {train_acc:.4f}, Train Loss: {train_loss:.4f}")

test_loss, test_acc = evaluate_model(model, [X_img_test, X_feat_test], y_test, model_type="Transfer Learning")
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

Train Accuracy: 0.6382, Train Loss: 0.6945
133/133 ━━━━━━━━━━━━━━━━━━━━ 12s 83ms/step - accuracy: 0.7024 - loss: 0.6162
Test Accuracy: 0.7014, Test Loss: 0.6129
